In [3]:
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn

from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

In [4]:
pds.set_option('display.max.colwidth', 50)
data_path = os.path.join('data')

train_df = pds.read_csv(os.path.join(data_path,'train.csv'))
validation_df = pds.read_csv(os.path.join(data_path,'validation_sample.csv'))
test_df = pds.read_csv(os.path.join(data_path,'test.csv'))
submission_df = pds.read_csv(os.path.join(data_path,'sample_submission.csv'))
train_df.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [42]:
pds.set_option('display.max.colwidth', 800)
train_df.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
1,1,0,"Feb 8 16:21:00 localhost logstash: [2021-02-08T16:21:00,548][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[wazuh-alerts-audit-3.x-2021.16.08][3] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[wazuh-alerts-audit-3.x-2021.16.08][3]] containing [25] requests]""})"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
4,4,1,"type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm=""date"" exe=""/usr/bin/date"" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603094402.016:52981): cwd=""/root"" type=PATH msg=audit(1603094402.016:52981): item=0 name=""/etc/ld.so.cache"" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564"


In [54]:
len(train_df.level==0)

472972

In [40]:
train_df.loc[
    train_df.full_log.str.contains('kibana:')
]

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
6,6,0,"Jan 22 06:28:59 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-21T21:28:59Z"",""tags"":[""warning"",""stats-collection""],""pid"":4485,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
8,8,0,"Jan 20 08:16:43 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T23:16:43Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
...,...,...,...
472961,472961,0,"Jan 20 06:28:32 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T21:28:32Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
472965,472965,0,"Dec 28 15:35:05 localhost kibana: {""type"":""log"",""@timestamp"":""2020-12-28T06:35:05Z"",""tags"":[""error"",""task_manager""],""pid"":3816,""message"":""Failed to poll for work: Error: No Living connections""}"
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log"",""@timestamp"":""2020-10-11T17:20:29Z"",""tags"":[""error"",""task_manager""],""pid"":821,""message"":""Failed to poll for work: Erro

In [51]:
train_df.loc[
    (train_df.level==0) & ( train_df.full_log.str.contains('kibana:'))
]

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-24T01:02:22Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-12T16:50:40Z"",""tags"":[""warning"",""stats-collection""],""pid"":4332,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-04T01:18:31Z"",""tags"":[""warning"",""stats-collection""],""pid"":4820,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
6,6,0,"Jan 22 06:28:59 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-21T21:28:59Z"",""tags"":[""warning"",""stats-collection""],""pid"":4485,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
8,8,0,"Jan 20 08:16:43 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T23:16:43Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
...,...,...,...
472961,472961,0,"Jan 20 06:28:32 localhost kibana: {""type"":""error"",""@timestamp"":""2021-01-19T21:28:32Z"",""tags"":[""warning"",""stats-collection""],""pid"":9502,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
472965,472965,0,"Dec 28 15:35:05 localhost kibana: {""type"":""log"",""@timestamp"":""2020-12-28T06:35:05Z"",""tags"":[""error"",""task_manager""],""pid"":3816,""message"":""Failed to poll for work: Error: No Living connections""}"
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log"",""@timestamp"":""2020-10-11T17:20:29Z"",""tags"":[""error"",""task_manager""],""pid"":821,""message"":""Failed to poll for work: Erro

In [123]:
train_df.loc[
    (train_df.level!=0) & ( train_df.full_log.str.contains('kibana:'))
]

,id,level,full_log


In [121]:
train_df.loc[
    (train_df.level==0) & (train_df.full_log.str.contains('logstash:')==False) & ( train_df.full_log.str.contains('kibana:')==False) & ( train_df.full_log.str.contains('mySQL port')==False) &
    ( train_df.full_log.str.contains('esild-ml-start.sh:')==False) & ( train_df.full_log.str.contains('probe_rpminfo:')==False) & ( train_df.full_log.str.contains('mcelog:')==False) &
    ( train_df.full_log.str.contains('GRES')==False) & ( train_df.full_log.str.contains('Packages')==False) & ( train_df.full_log.str.contains('reached -')==False) &
    ( train_df.full_log.str.contains('exresnte')==False) & ( train_df.full_log.str.contains('journal:')==False)
]

,id,level,full_log
562,562,0,"level : 5, log : No mode specified for interface device interface-name; defaulting to port mode"
992,992,0,"error: db5 error(-30973) from dbenv->failchk: BDB0087 DB_RUNRECOVERY: Fatal error, run database recovery"
1390,1390,0,"level : 5, log : Error getting class usage statistics for interface interface-name.unit-id: error-message"
1569,1569,0,"level : 5, log : Fabric fc-fabric-name interface interface-name mac mac-address deleted Description Fibre channel over Ethernet logical interface is deleted."
1663,1663,0,"level : 5, log : Failed to parse community string under policy-options: error-message"
...,...,...,...
470486,470486,0,juniper
470540,470540,0,"level : 5, log : Unable to create interface devices during attachment of PIC pic-slot in FPC fpc-slot: graceful switchover not supported"
471241,471241,0,"level : 3, log : routing socket open error argument"
471457,471457,0,"level : 10, log : Symmetric key generation failure: error-message"


In [6]:
train_df.loc[
    (train_df.level!=0) & ( train_df.full_log.str.contains('error-message'))
]

,id,level,full_log
3538,3538,1,"level : 5, log : error-message: lacp current while timer expired current Receive State: CURRENT"
6178,6178,1,"level : 5, log : Unable to generate OID from product-name (error-message)"
7168,7168,1,"level : 5, log : Unable to delete CoS profile (L2TP tunnel tunnel-id, session session-id): error-message"
8607,8607,1,"level : 5, log : Ignoring RADIUS server radius-server: unable to bind to source address source-address (error-message)"
10045,10045,1,"level : 5, log : function-name operation failed for object-name: index1 index1 index2 index2 index3 index3 index4 index4 (error-message)"
...,...,...,...
460259,460259,1,"level : 3, log : Error:error-message"
464701,464701,1,"level : 3, log : Message: error-message"
465000,465000,1,"level : 5, log : Unable to create event context: error-message"
466741,466741,1,"level : 5, log : function-name: write error on pipe to ppmd (error-message)"


In [9]:
train_df.loc[train_df.level==1]

,id,level,full_log
4,4,1,"type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm=""date"" exe=""/usr/bin/date"" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603094402.016:52981): cwd=""/root"" type=PATH msg=audit(1603094402.016:52981): item=0 name=""/etc/ld.so.cache"" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564"
5,5,1,"type=SYSCALL msg=audit(1611889244.855:247124): arch=c000003e syscall=2 success=yes exit=3 a0=7f1c14d535a4 a1=80000 a2=1 a3=7f1c14f594f8 items=1 ppid=100038 pid=100039 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm=""ps"" exe=""/usr/bin/ps"" subj=system_u:system_r:unconfined_service_t:s0 key=""audit-wazuh-r"" type=CWD msg=audit(1611889244.855:247124): cwd=""/"" type=PATH msg=audit(1611889244.855:247124): item=0 name=""/etc/ld.so.cache"" inode=35395307 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611889244.855:247124): proctitle=2F62696E2F7073002D700032353336"
10,10,1,"type=SYSCALL msg=audit(1611895525.367:489785): arch=c000003e syscall=59 success=yes exit=0 a0=14de250 a1=14de360 a2=14dd150 a3=7ffd3604d7a0 items=2 ppid=7383 pid=7384 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm=""ps"" exe=""/usr/bin/ps"" subj=system_u:system_r:unconfined_service_t:s0 key=""audit-wazuh-x"" type=EXECVE msg=audit(1611895525.367:489785): argc=3 a0=""/bin/ps"" a1=""-p"" a2=""2514"" type=CWD msg=audit(1611895525.367:489785): cwd=""/"" type=PATH msg=audit(1611895525.367:489785): item=0 name=""/bin/ps"" inode=101254309 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:bin_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PATH msg=audit(1611895525.367:489785): item=1 name=""/lib64/ld-linux-x86-64.so.2"" inode=80122 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:ld_so_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe..."
15,15,1,"type=SYSCALL msg=audit(1603072847.573:38657): arch=c000003e syscall=2 success=yes exit=8 a0=7fa584948547 a1=80000 a2=1b6 a3=24 items=1 ppid=1 pid=30216 auid=0 uid=0 gid=985 euid=0 suid=0 fsuid=0 egid=985 sgid=985 fsgid=985 tty=(none) ses=1 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603072847.573:38657): cwd=""/"" type=PATH msg=audit(1603072847.573:38657): item=0 name=""/etc/group"" inode=68678689 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:passwd_file_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603072847.573:38657): proctitle=""/var/esild/bin/ossec-syscheckd"""
22,22,1,"type=SYSCALL msg=audit(1603159410.494:164830): arch=c000003e syscall=2 success=yes exit=8 a0=6cbc60 a1=0 a2=1b6 a3=24 items=1 ppid=1 pid=30216 auid=0 uid=0 gid=985 euid=0 suid=0 fsuid=0 egid=985 sgid=985 fsgid=985 tty=(none) ses=1 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key=""audit-wazuh-r"" type=CWD msg=audit(1603159410.494:164830): cwd=""/"" type=PATH msg=audit(1603159410.494:164830): item=0 name=""/bin/gucharmap"" inode=1140299 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:bin_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603159410.494:164830): proctitle=""/var/esild/bin/ossec-syscheckd"""
...,...,...,...
472948,472948,1,Oct

In [52]:
train_df.loc[
    (train_df.level!=1) & ( train_df.full_log.str.contains('esild'))
]

,id,level,full_log
16,16,0,Jan 22 08:07:15 localhost esild-ml-start.sh: Unable to open file (file read failed: time = Fri Jan 22 08:07:15 2021
52,52,0,"Jan 22 07:40:31 localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = 0x7fff8eef2a48, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)"
337,337,3,Jan 22 11:37:57 localhost systemd: Unit esild-ml.service entered failed state.
368,368,0,"Jan 22 06:03:58 localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = 0x7ffcac69cc98, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)"
572,572,0,Jan 22 11:58:31 localhost esild-ml-start.sh: Unable to open file (file read failed: time = Fri Jan 22 11:58:31 2021
...,...,...,...
472542,472542,0,"Jan 22 07:17:48 localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = 0x7ffe344294c8, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)"
472563,472563,0,"Jan 22 13:26:32 localhost esild-ml-start.sh: , filename = '/var/esild/ml/model1', file descriptor = 3, errno = 21, error message = 'Is a directory', buf = 0x7ffc46ea53b8, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)"
472668,472668,0,Jan 22 02:36:23 localhost esild-ml-start.sh: Unable to open file (file read failed: time = Fri Jan 22 02:36:23 2021
472770,472770,0,Jan 22 06:30:21 localhost esild-ml-start.sh: Unable to open file (file read failed: time = Fri Jan 22 06:30:21 2021


In [35]:
train_df.loc[
    (train_df.level==1) & ( train_df.full_log.str.contains('fsgid')==False) & ( train_df.full_log.str.contains('root :')==False) & ( train_df.full_log.str.contains('sudo:session')==False)
]

,id,level,full_log
189,189,1,Dec 15 16:52:49 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/agents ; USER=root ; COMMAND=/var/esild/bin/agent_control -i 021 sed s/Agent Name://
334,334,1,System Audit: CIS - RHEL7 - 4.1.1 - Network parameters - IP Forwarding enabled {CIS: 4.1.1 RHEL7} {PCI_DSS: 2.2.4}. File: /proc/sys/net/ipv4/ip_forward. Reference: https://benchmarks.cisecurity.org/tools2/linux/CIS_Red_Hat_Enterprise_Linux_7_Benchmark_v1.1.0.pdf .
482,482,1,System Audit: CIS - RHEL7 - 1.4.4 - SELinux setroubleshoot enabled {CIS: 1.4.4 RHEL7} {PCI_DSS: 2.2.4}. File: /usr/share/dbus-1/services/sealert.service. Reference: https://benchmarks.cisecurity.org/tools2/linux/CIS_Red_Hat_Enterprise_Linux_7_Benchmark_v1.1.0.pdf .
622,622,1,System Audit: SSH Hardening - 3: Root can log in. File: /etc/ssh/sshd_config. Reference: 3 .
830,830,1,Jan 8 14:55:28 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/cat /var/esild/ruleset/decoders/0379-dpkg_decoders.xml
...,...,...,...
472340,472340,1,Feb 15 13:38:59 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/cat /var/esild/ruleset/rules/0235-vmware_rules.xml
472469,472469,1,Jan 25 14:25:50 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/cat /var/esild/ruleset/rules/0110-ms_dhcp_rules.xml
472627,472627,1,System Audit: SSH Hardening - 9: Wrong Maximum number of authentication attempts {PCI_DSS: 2.2.4}. File: /etc/ssh/sshd_config. Reference: 9 .
472894,472894,1,Dec 16 14:35:27 localhost sshd[18446]: pam_unix(sshd:session): session opened for user root by (uid=0)
